In [1]:
import librosa
import numpy as np
import struct

# Main function
file_path = "teste.mp3"  # Replace this with your MP3 file path
audio, sr = librosa.load(file_path, sr=None)
def float_to_binary(f):
    # Convert float to bytes
    b = struct.pack('f', f)
    # Convert bytes to binary string
    return ''.join(f'{x:08b}' for x in b)

def binary_to_float(binary):
    # Convert binary string to bytes
    b = bytes(int(binary[i:i+8], 2) for i in range(0, len(binary), 8))
    # Unpack bytes to float
    return struct.unpack('f', b)[0]


In [7]:
bin_vector = []
i=0
for value in audio:
    binary = float_to_binary(value)
    for i in range(len(binary)):
        if binary[i] == '0':
            bin_vector.append(-2)
            bin_vector.append(-2)
            bin_vector.append(-2)
        else:
            bin_vector.append(2)
            bin_vector.append(2)
            bin_vector.append(2)
ruido = np.random.normal(0,0.8, len(bin_vector))

bin_vector = bin_vector + ruido

In [8]:
import math

pi_const = 3.141592653589793238462643383279

A = 1
#frequencia
w = 10e6
#quantidade de amostras
samples = len(bin_vector)
#constante da sensiblidade da amplitude do modulador
k = 0.8
#vetor vazio q vai receber as amostras
sender = []
# Transform character to bytes
step = 1/(int(samples/4))
for i in range(samples):
    # Transform bytes to integer
    t = i*step
    msg = bin_vector[i]
    sender.append(A*(1 + msg*k)*math.cos(w*t*2*pi_const))

In [9]:
receiver = []
i = 0

while i< samples:
    t = i*step
    elem1 = ((sender[i]/(math.cos(w*t*2*pi_const)*A)) - 1)/k
    elem2 = ((sender[i+1]/(math.cos(w*(t+ 1*step)*2*pi_const)*A)) - 1)/k
    elem3 = ((sender[i+2]/(math.cos(w*(t+ 2*step)*2*pi_const)*A)) - 1)/k
    sum = elem1 + elem2 + elem3
    if sum > 0:
        receiver.append(1)
    else:
        receiver.append(0)
    i+=3

In [10]:
import soundfile as sf

# Your code for converting binary data to a list of floats
float_message = []
for i in range(0, len(receiver), 32):
    # Convert the binary array to a string
    binary_string = ''.join(str(bit) for bit in receiver[i:i + 32])
    # Convert the binary string to a float
    decimal_value = binary_to_float(binary_string)
    # Append the float value to the list
    float_message.append(decimal_value)
# Save the list of floats as an audio file
sf.write('teste.wav', np.array(float_message), sr)
